In [13]:
from keras.layers import Input, Dense, TimeDistributed, Embedding
from keras.layers import Concatenate, Reshape, Lambda, Multiply, multiply, concatenate
from keras.models import Model
from keras import backend as K

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = ''

import tensorflow as tf
import numpy as np

In [14]:
# make first model

def build_base_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    tanh_output = Dense(1, activation='tanh', name='tanh_output')(input_layer)
    
    model = Model(inputs=input_layer, outputs=tanh_output)
    model.summary()
    return model

In [15]:
# load data
# make sure that the first shape is the IMDB training data. 

def open_pickle(path):
    import pickle
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train_original = open_pickle('../data/imdb/imdb_original_preprocessed_xtrain.pickle')
X_test_original = open_pickle('../data/imdb/imdb_original_preprocessed_xtest.pickle')
y_train_original = open_pickle('../data/imdb/imdb_original_preprocessed_ytrain.pickle')
y_test_original = open_pickle('../data/imdb/imdb_original_preprocessed_ytest.pickle')

In [16]:
# Count vectorizer 

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df = 100)
X_train = cv.fit_transform(X_train_original)
X_test = cv.transform(X_test_original)

In [17]:
def load_unigrams(path, X, y):
    word_list = []
    connotation = {}
    
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            word_list.append(line.strip())
            
    for word in word_list:
        pos_count = 0
        neg_count = 0
        for i, doc in enumerate(X):
            if word in doc.lower():
                if (y[i] == 1):
                    pos_count += 1
                else:
                    neg_count += 1
                    
        if pos_count > neg_count:
            connotation[word] = 1
        else:
            connotation[word] = 0
    
    return word_list, connotation

def generate_appearance(X_train_corpus, X_test_corpus, word_list, connotation):
    y_train_agreement = []
    for i in range(len(X_train_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_train_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_train_agreement.append(doc_agreement)
        
    y_test_agreement = []
    for i in range(len(X_test_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_test_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_test_agreement.append(doc_agreement)
        
    return np.array(y_train_agreement), np.array(y_test_agreement)

# 'imdb-unigrams.txt'

In [18]:
word_list, connotation = load_unigrams('./imdb-unigrams.txt', X_train_original, y_train_original)

In [19]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_original, X_test_original, 
                                                          word_list, connotation)

In [20]:
# build the combined model
# Combined model
human_terms_len = len(word_list)

base_model = build_base_model(X_train.shape[1])

combined_input_layer = Input(shape=(X_train.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

# get the document prediction
label_layer = base_model(combined_input_layer)

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
    dense_layer.append(Dense(1, activation='relu',use_bias=False, kernel_initializer='ones')(Multiply()([split[i], label_layer])))

# concat all the result   
concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 3641)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3642      
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 83)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 3641)         0                                            
_________________________________________

In [21]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [22]:
y_train_tanh = y_train_original
y_train_tanh[y_train_tanh == 0] = -1

In [23]:
y_train_agreement.shape

(25000, 83)

In [24]:
base_model_history = base_model.fit(X_train[:16667], y_train_tanh[:16667], 
                                    validation_data=(X_train[16667:], y_train_tanh[16667:]),
                                    batch_size=1, epochs=10)

Train on 16667 samples, validate on 8333 samples
Epoch 1/10
16667/16667 [==============================] - 52s 3ms/step - loss: 0.4535 - acc: 0.6938 - val_loss: 0.4071 - val_acc: 0.7761
Epoch 2/10
16667/16667 [==============================] - 50s 3ms/step - loss: 0.3211 - acc: 0.8247 - val_loss: 0.4571 - val_acc: 0.7954
Epoch 3/10
16667/16667 [==============================] - 50s 3ms/step - loss: 0.2812 - acc: 0.8589 - val_loss: 0.4226 - val_acc: 0.8103
Epoch 4/10
16667/16667 [==============================] - 50s 3ms/step - loss: 0.2530 - acc: 0.8790 - val_loss: 0.4363 - val_acc: 0.8154
Epoch 5/10
16667/16667 [==============================] - 50s 3ms/step - loss: 0.2314 - acc: 0.8856 - val_loss: 0.4410 - val_acc: 0.8175
Epoch 6/10
16667/16667 [==============================] - 50s 3ms/step - loss: 0.2178 - acc: 0.8980 - val_loss: 0.4516 - val_acc: 0.8226
Epoch 7/10
16667/16667 [==============================] - 39s 2ms/step - loss: 0.2017 - acc: 0.9069 - val_loss: 0.4616 - val_acc:

In [25]:
combined_model_history = combined_model.fit([X_train,y_train_agreement], y_train_original, batch_size=1, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 81s 3ms/step - loss: 0.7278 - acc: 0.3520
Epoch 2/10
25000/25000 [==============================] - 90s 4ms/step - loss: 0.6801 - acc: 0.3774
Epoch 3/10
25000/25000 [==============================] - 81s 3ms/step - loss: 0.6786 - acc: 0.3819
Epoch 4/10
25000/25000 [==============================] - 95s 4ms/step - loss: 0.6755 - acc: 0.3825
Epoch 5/10
25000/25000 [==============================] - 85s 3ms/step - loss: 0.6773 - acc: 0.3779: 0s - lo
Epoch 6/10
25000/25000 [==============================] - 89s 4ms/step - loss: 0.6789 - acc: 0.3819
Epoch 7/10
25000/25000 [==============================] - 95s 4ms/step - loss: 0.6775 - acc: 0.3802
Epoch 8/10
25000/25000 [==============================] - 85s 3ms/step - loss: 0.6782 - acc: 0.3764
Epoch 9/10
25000/25000 [==============================] - 89s 4ms/step - loss: 0.6796 - acc: 0.3798
Epoch 10/10
25000/25000 [==============================] - 97s 4ms/step - loss: 0.6795 - ac

In [26]:
# independent weight here
# independent_weight_base_model_history
# independent_weight_combined_model_history

In [27]:
# var name
# shared_weight_base_model_history
# shared_weight_combined_model_history

In [28]:
net_weight = base_model.get_weights()

In [29]:
len(net_weight)

2

In [30]:
weight = net_weight[0].flatten()

In [31]:
net_weight[1].shape

(1,)

In [32]:
words = cv.get_feature_names()

In [33]:
indices = np.argsort(weight)

In [34]:
for i in indices[:10]:
    print('%s \t %.3f' %(words[i], weight[i]))

worst 	 -3.291
dull 	 -2.936
waste 	 -2.751
incoherent 	 -2.689
unwatchable 	 -2.626
fails 	 -2.481
behave 	 -2.347
mst3k 	 -2.274
awful 	 -2.269
disappointing 	 -2.229


In [35]:
pos_indices = indices[::-1]
for i in pos_indices[:10]:
    print('%s \t %.3f' %(words[i], weight[i]))

refreshing 	 2.957
flawless 	 2.936
rare 	 2.623
excellent 	 2.609
complaint 	 2.476
gem 	 2.276
unforgettable 	 2.245
captures 	 2.240
touching 	 2.236
surprised 	 2.212


In [36]:
combined_net_weight = combined_model.get_weights()

In [37]:
len(combined_net_weight)

87

In [38]:
ht_weight = []
for i in range(len(combined_net_weight)):
    if i>1 and i < 85:
        print(combined_net_weight[i].flatten())
        ht_weight.append(combined_net_weight[i].flatten())
    else:
        print(i, combined_net_weight[i].shape)
ht_weight = np.asarray(ht_weight)

0 (3641, 1)
1 (1,)
[3.856292]
[3.785044]
[3.9499063]
[4.5709133]
[1.5295228]
[0.79288]
[4.017805]
[4.1203136]
[3.5567083]
[3.733695]
[2.9015322]
[3.905837]
[3.9975376]
[4.4618635]
[3.8926506]
[2.2900755]
[2.9117887]
[2.2432423]
[2.7752683]
[3.844108]
[3.8464253]
[3.8587027]
[3.9292893]
[4.028082]
[3.9974139]
[4.139554]
[4.102579]
[3.8500907]
[3.0146685]
[2.7355192]
[3.0281527]
[4.1785393]
[3.0539448]
[3.528832]
[3.4079034]
[3.778395]
[2.6149385]
[3.4729388]
[2.507975]
[2.792424]
[2.6840122]
[4.1161113]
[3.3761716]
[4.1283383]
[4.2900105]
[3.7933712]
[4.309934]
[3.9420862]
[2.8144515]
[4.015683]
[3.2209988]
[3.2406428]
[3.9150593]
[3.7406783]
[3.91127]
[3.037793]
[2.400901]
[4.363614]
[4.3143983]
[2.928465]
[3.4772673]
[3.296382]
[3.010942]
[3.8090694]
[3.5298448]
[4.1824927]
[3.5807912]
[3.4150798]
[4.139684]
[3.318446]
[3.4232268]
[3.3166075]
[3.901336]
[4.4739785]
[3.9725366]
[2.9061453]
[3.9075885]
[2.620676]
[3.9696388]
[2.8973053]
[1.8304212]
[3.8868985]
[4.164777]
85 (83, 1)
86 (

In [39]:
sigmoid_weight = combined_net_weight[85].flatten()

In [40]:
ht_weight = ht_weight.flatten()

In [41]:
human_term_indices = np.argsort(ht_weight)

print('human terms \t dense_relu weight \t sigmoid weight \t\t pos/neg?')
for i in human_term_indices:
    print('%s \t %.3f \t %.3f \t\t %.1f' %(word_list[i], ht_weight[i], sigmoid_weight[i], connotation[word_list[i]]))

human terms 	 dense_relu weight 	 sigmoid weight 		 pos/neg?
6/10 	 0.793 	 -0.019 		 1.0
5/10 	 1.530 	 0.712 		 1.0
wonderfully 	 1.830 	 0.833 		 1.0
beautifully 	 2.243 	 1.358 		 1.0
badly 	 2.290 	 -1.337 		 0.0
perfectly 	 2.401 	 1.472 		 1.0
funniest 	 2.508 	 1.483 		 1.0
fun 	 2.615 	 1.792 		 1.0
wasted 	 2.621 	 -1.670 		 0.0
great 	 2.684 	 1.740 		 1.0
enjoyed 	 2.736 	 1.958 		 1.0
best 	 2.775 	 1.809 		 1.0
gem 	 2.792 	 1.869 		 1.0
loved 	 2.814 	 1.875 		 1.0
wonderful 	 2.897 	 1.969 		 1.0
amazing 	 2.902 	 2.069 		 1.0
unfunny 	 2.906 	 -2.097 		 0.0
beautiful 	 2.912 	 1.987 		 1.0
poorly 	 2.928 	 -2.188 		 0.0
recommended 	 3.011 	 2.039 		 1.0
enjoyable 	 3.015 	 1.899 		 1.0
excellent 	 3.028 	 2.213 		 1.0
perfect 	 3.038 	 2.104 		 1.0
fantastic 	 3.054 	 2.002 		 1.0
mess 	 3.221 	 -2.309 		 0.0
mst3k 	 3.241 	 -2.161 		 0.0
rare 	 3.296 	 2.284 		 1.0
surprisingly 	 3.317 	 2.468 		 1.0
subtle 	 3.318 	 2.388 		 1.0
incredible 	 3.376 	 2.486 		 1.0
fav

In [42]:
len(connotation)

83

In [43]:
# Print report on the word transparency
index = [9, 19]
def report():
    for i in index:
        print()
        bm = base_model.predict(X_test[i])
        
        cm = combined_model.predict([X_test[i], 
                                np.reshape(y_test_agreement[i], (1,y_test_agreement.shape[1]))])
        
#         document_output = 'multiply'
#         document_predict = Model(inputs=combined_model.input,
#                                      outputs=combined_model.get_layer(document_output).output)
#         doc_output = document_predict.predict([np.reshape(data[i], (1,5)), 
#                                       ht_1_input[i], 
#                                       ht_2_input[i], 
#                                       ht_3_input[i], 
#                                       ht_4_input[i]])
        
        layer_name = 'concatenate'
        concat_after_relu = Model(inputs=combined_model.input,
                                     outputs=combined_model.get_layer(layer_name).output)
        concat_output = concat_after_relu.predict([X_test[i], 
                                np.reshape(y_test_agreement[i], (1,y_test_agreement.shape[1]))])
        
        print(X_test_original[i], '\n\n actual label : ', y_test_original[i], '\n predict from base model : ', bm.flatten(), '\n predict label : ', cm.flatten())
    
        for i,output in enumerate(concat_output.flatten()):
            if output != 0:
                print(word_list[i], output)

In [44]:
report()


hilarious, clean, light-hearted, and quote-worthy. what else can you ask for in a film? this is my all-time, number one favorite movie. ever since i was a little girl, i have dreamed of owning a blue van with flame and an observation bubble.the cliché character in ridiculous situation are what make this film such great fun. the wonderful comedic chemistry between stephen furst (harold) and andy tennant (melio) make up most of my favorite part of the movie. and who did not love the hopeless awkwardness of flynch? do not forget the airport antic of leon's crony, dressed up as hari krishna: dancing, chanting and playing the tambourine--unbeatable! the clue are genius, the location are classic, and the plot is timeless.a word to the wise, if you did not watch this film when you were little, it probably will not win a place in your heart today. but nevertheless give it a chance, you may find that "it does not matter what you say, it does not matter what you do, you have gotta play." 

 act

In [45]:
report()


hilarious, clean, light-hearted, and quote-worthy. what else can you ask for in a film? this is my all-time, number one favorite movie. ever since i was a little girl, i have dreamed of owning a blue van with flame and an observation bubble.the cliché character in ridiculous situation are what make this film such great fun. the wonderful comedic chemistry between stephen furst (harold) and andy tennant (melio) make up most of my favorite part of the movie. and who did not love the hopeless awkwardness of flynch? do not forget the airport antic of leon's crony, dressed up as hari krishna: dancing, chanting and playing the tambourine--unbeatable! the clue are genius, the location are classic, and the plot is timeless.a word to the wise, if you did not watch this film when you were little, it probably will not win a place in your heart today. but nevertheless give it a chance, you may find that "it does not matter what you say, it does not matter what you do, you have gotta play." 

 act

In [46]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)

25000/25000 [==============================] - 1s 56us/step


In [47]:
score

[0.17616602549791335, 0.80992]

### Let's see the weight when the trainable is false

In [113]:
def build_combined_model():
    # build the combined model
    # Combined model
    human_terms_len = len(word_list)

    base_model = build_base_model(X_train.shape[1])

    combined_input_layer = Input(shape=(X_train.shape[1],))

    # build the hard coded weight for human terms
    ht_input_layer = Input(shape=(human_terms_len,))

    split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

    # get the document prediction
    label_layer = base_model(combined_input_layer)

    # stack the multiply layer
    dense_layer = []
    for i in range(human_terms_len):
        dense_layer.append(Dense(1, activation='relu',use_bias=False, kernel_initializer='ones', trainable=True)(Multiply()([split[i], label_layer])))

    # concat all the result   
    concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

    # pass it to sigmoid layer
    output_layer = Dense(1, activation='sigmoid')(concat)

    combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
    combined_model.summary()
    
    return base_model, combined_model

In [114]:
base_model, combined_model = build_combined_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 3641)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3642      
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 83)           0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 3641)         0                                            
_________________________________________

In [115]:
base_model.compile(loss='mse',
                  optimizer='Adagrad',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='Adagrad',
                      metrics=['acc'])

In [116]:
base_model_history = base_model.fit(X_train[:16667], y_train_tanh[:16667], 
                                    validation_data=(X_train[16667:], y_train_tanh[16667:]),
                                    batch_size=1, epochs=50)

Train on 16667 samples, validate on 8333 samples
Epoch 1/50
16667/16667 [==============================] - 35s 2ms/step - loss: 0.4916 - acc: 0.5315 - val_loss: 0.4258 - val_acc: 0.5970
Epoch 2/50
16667/16667 [==============================] - 33s 2ms/step - loss: 0.3561 - acc: 0.6535 - val_loss: 0.4005 - val_acc: 0.6403
Epoch 3/50
16667/16667 [==============================] - 33s 2ms/step - loss: 0.3231 - acc: 0.6857 - val_loss: 0.3883 - val_acc: 0.6570
Epoch 4/50
16667/16667 [==============================] - 33s 2ms/step - loss: 0.3032 - acc: 0.7049 - val_loss: 0.3820 - val_acc: 0.6714
Epoch 5/50
16667/16667 [==============================] - 33s 2ms/step - loss: 0.2896 - acc: 0.7185 - val_loss: 0.3812 - val_acc: 0.6780
Epoch 6/50
16667/16667 [==============================] - 33s 2ms/step - loss: 0.2795 - acc: 0.7283 - val_loss: 0.3750 - val_acc: 0.6829
Epoch 7/50
16667/16667 [==============================] - 33s 2ms/step - loss: 0.2712 - acc: 0.7364 - val_loss: 0.3730 - val_acc:

In [117]:
combined_model_history = combined_model.fit([X_train[:16667],y_train_agreement[:16667]], y_train_original[:16667], batch_size=1, epochs=50)

Epoch 1/50
16667/16667 [==============================] - 71s 4ms/step - loss: 0.9014 - acc: 0.2008
Epoch 2/50
16667/16667 [==============================] - 65s 4ms/step - loss: 0.7652 - acc: 0.3133
Epoch 3/50
16667/16667 [==============================] - 71s 4ms/step - loss: 0.7285 - acc: 0.3335
Epoch 4/50
16667/16667 [==============================] - 64s 4ms/step - loss: 0.7090 - acc: 0.3458
Epoch 5/50
16667/16667 [==============================] - 73s 4ms/step - loss: 0.6962 - acc: 0.3534
Epoch 6/50
16667/16667 [==============================] - 68s 4ms/step - loss: 0.6867 - acc: 0.3588
Epoch 7/50
16667/16667 [==============================] - 72s 4ms/step - loss: 0.6792 - acc: 0.3637
Epoch 8/50
16667/16667 [==============================] - 64s 4ms/step - loss: 0.6732 - acc: 0.3667
Epoch 9/50
16667/16667 [==============================] - 73s 4ms/step - loss: 0.6683 - acc: 0.3697
Epoch 10/50
16667/16667 [==============================] - 64s 4ms/step - loss: 0.6642 - acc: 0.3715

In [109]:
# base_model_history_trainfalse = base_model_history
# combined_model_history_trainfalse = combined_model_history

In [118]:
net_weights = combined_model.get_weights()

In [ ]:
# Evaluate model

### Generate color weighted

In [119]:
import re

class ColoredWeightedDoc(object):
    def __init__(self, doc, human_terms, ht_weights, token_pattern=r"(?u)\b\w\w+\b", binary = False):
        self.doc = doc
        self.human_terms = human_terms
        self.ht_weights = ht_weights
        self.binary = binary
        self.tokenizer = re.compile(token_pattern)
#         self.abs_ranges = np.linspace(0, max([abs(coefs.min()), abs(coefs.max())]), 8)
    def _repr_html_(self):
        html_rep = ""
        tokens = self.doc.split(" ") 
        if self.binary:
            seen_tokens = set()       
        for token in tokens:
            vocab_tokens = self.tokenizer.findall(token.lower())
            if len(vocab_tokens) > 0:
                vocab_token = vocab_tokens[0]
                try:
                    vocab_index = self.human_terms.index(vocab_token)
                    
                    if not self.binary or vocab_index not in seen_tokens:
                        
                        if self.ht_weights[vocab_index] == 0: # human-terms which has been washed out (opposing)
                            html_rep = html_rep + "<font size = 2, color=lightgreen> " + token + " </font>"
                        
                        elif self.ht_weights[vocab_index] != 0: # human-terms transparency
                            html_rep = html_rep + "<font size = 3, color=blue> " + token + " </font>"
                        
                        else: # neutral word
                            html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
                        
                        if self.binary:    
                            seen_tokens.add(vocab_index)
                    
                    else: # if binary and this is a token we have seen before
                        html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
                except: # this token does not exist in the vocabulary
                    html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
            else:
                html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
        return html_rep

In [177]:
from IPython import display
idx = 1001
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

actual label :  0 
predict from base model :  [-1.] 
predict label :  [3.042287e-08]

awful 2.2876508
boring 2.2876978
horrible 2.114616
pathetic 2.090236
ridiculous 2.13441
stupid 2.4445555
waste 2.5845466


In [163]:
np.where(np.sum(y_test_agreement, axis=1)!=0)[0][1000:1010]

array([1231, 1232, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241],
      dtype=int64)

In [123]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)
score

25000/25000 [==============================] - 2s 83us/step


[0.13361379306793214, 0.82476]

In [124]:
concat_all = concat_after_relu.predict([X_test, y_test_agreement])

In [125]:
concat_all.shape

(25000, 83)

In [145]:
indices = np.where(np.sum(concat_all, axis=1)!=0)
indices = indices[0]
len(indices)

21909

In [148]:
not_zero_predict = combined_model.evaluate([X_test[indices], y_test_agreement[indices]], y_test_original[indices])

21909/21909 [==============================] - 2s 72us/step


In [149]:
not_zero_predict

[0.09822631518490532, 0.8798210781175242]